**This code example works with raw MS Data downloaded from the MetaboLights repository (~ 1 GiB). The running time for the code in this notebook is approximately 30 s (excluding the time required to download the data), using a personal computer with an 8th generation Intel i5 processor and 8 GiB memory.**

# Application 1: System suitability check, and signal drift evaluation

This notebook introduces the analysis of the application #1 published in [Metabolites](https://doi.org/10.3390/metabo10100416). It shows how to work with raw data using as an example a System Suitability Check conducted in a metabolomics experiment: System Suitability Samples (SSS) were prepared using five known chemical standards:

* Alogliptin
* Phe-Phe
* Tryptophan
* LPC 18:0
* Leu-Enk

Ten SSS samples (addressed as SSS1) were consecutively run and used to build an acceptance criteria, and then compared against values obtained from the analysis of SSS samples that were analyzed before (SSS2) and after (SSS3) the study samples. This analysis is displayed in Figure 3.

A similar analysis was conducted using QC samples that were spiked with the same compounds and with Leu-13C used as internal standard, but in this case, no acceptance criteria was defined. These results are displayed in Figure S1.


<img src="fig/sample-list.png" width=700>
Sample list used in the experiment. The first SSS sample is the SSS2 and the second SSS sample is SSS3. SSS1 is not shown in this figure.

**UPDATE 2022-06**: The code has been modified to work with the Assay object. See [this link](agregar) for a description on how to work with Assay objects.

In [ ]:
import tidyms as ms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from download_from_metabolights import get_application1_data
pd.set_option("display.precision", 4)
sns.set_context("paper", font_scale=1.5)
import bokeh.plotting

## Loading data from Metabolights

In [ ]:
# Setting the raw data path and a DataFrame with sample metadata
data_path = "data"
get_application1_data(data_path)  # download data from metabolights
sample_list_path = os.path.join(data_path, "sample_list.csv")
centroid_data_path = os.path.join(data_path, "cent")
sample_list = pd.read_csv(sample_list_path)
# sample_list = sample_list[~(sample_list["class"] == "SCQC")]

In [ ]:
# compounds used for SSS check, with their m/z and expected retention times
d = {
    "Compound": ["Leu-13C", "Trp", "Phe-Phe", "Alogliptin", "LPC 18:0", "Leu-Enk"],
     "rt": np.array([75, 129, 320, 291, 775, 372]),
     "mz": np.array([133.1056, 205.0977,313.1552, 340.1773, 524.3716, 556.2771])
}
df = pd.DataFrame(data=d)

## Feature Detection

In [ ]:
# create assay
assay = ms.Assay(assay_path="sss", data_path="data/cent", sample_metadata=sample_list)

In [ ]:
%%time
# ROI creation
mz = np.array([133.1056, 205.0977,313.1552, 340.1773, 524.3716, 556.2771])
roi_params = {"targeted_mz": d["mz"], "min_intensity": 500, "tolerance": 0.015}
assay.detect_features(n_jobs=-1, **roi_params)

## Feature extraction

In [ ]:
# peak detection
assay.extract_features(n_jobs=-1, store_smoothed=True)

In [ ]:
# peak descriptors
assay.describe_features(n_jobs=-1)

## Feature correspondence

In [ ]:
# build feature table
assay.build_feature_table()

In [ ]:
assay.feature_table

In [ ]:
# feature correspondence
assay.match_features(include_classes=["QC"], verbose=True)

## Data matrix creation

In [ ]:
data = assay.make_data_matrix()

In [ ]:
data.sample_metadata

In [ ]:
data.data_matrix

In [ ]:
data.feature_metadata

## Remove unwanted features using the Retention times from the standards

In [ ]:
# find the compounds used in the SSS using the expected rt 
ft_to_compound = dict()
for index in df.index:
    mz = df.at[index, "mz"]
    rt = df.at[index, "rt"]
    compound = df.loc[index, "Compound"]
    ft = data.select_features(mz, rt)
    ftid = int(ft[0].split("-")[-1])
    ft_to_compound[ftid] = compound

## Figure 3: m/z, Rt and area dispersion for SSS samples

In [ ]:
feature_table = assay.feature_table.copy()
feature_table["Compound"] = feature_table["cluster_"].map(ft_to_compound)

In [ ]:
# FIGURE 3: 

sss_mask = feature_table["class_"].isin(["SSS1", "SSS2", "SSS3"])
sss_data = feature_table[sss_mask].copy()
# sss_data = sss_data[sss_data["Compound"] != 'LPC 18:0']

# compute mean centered m/z and rt
mean_mz = sss_data["mz"].groupby(sss_data["Compound"]).mean()
mean_rt = sss_data["rt"].groupby(sss_data["Compound"]).mean()
sss_data["mean mz"] = \
    (sss_data["mz"].groupby(sss_data["Compound"])
     .apply(lambda x: x - mean_mz[x.name]))

sss_data["mean rt"] = \
    (sss_data["rt"].groupby(sss_data["Compound"])
     .apply(lambda x: x - mean_rt[x.name]))

xvars = ["mean mz", "mean rt", "area"]
g = sns.PairGrid(data=sss_data,
                 y_vars=["Compound"],
                 x_vars=xvars,
                 hue="class_",
                 hue_kws={"marker": [".", "X", "D"], "size": [8, 8, 8]},
                 height=4)
g.map(sns.stripplot)

# setting plot properties
g.axes[0, 0].set_xlim(-0.01, 0.01)
g.axes[0, 2].set_xticks(np.linspace(0, 2e5, 5))
t = g.axes[0, 2].get_xticks()
t = [str(x / 100000) for x in t ]
g.axes[0, 2].set_xticklabels(t);
g.axes[0, 0].set_xlabel("Mean centered m/z")
g.axes[0, 1].set_xlabel("Mean centered Rt [s]")
g.axes[0, 2].set_xlabel("Area / $10^{5}$ [au]");
# g.savefig("metabolomics-2020-sss.png", dpi=300)

 ## Figure S2: m/z, Rt and area dispersion for QC samples

In [ ]:
# FIGURE S1: 

# also remove LPC 18:0 because the area has much higher values
qc_mask = feature_table["class_"].isin(["QC"]) & (feature_table["Compound"] != "LPC 18:0")
qc_data = feature_table[qc_mask].copy()

# compute mean centered m/z and rt
mean_mz = qc_data["mz"].groupby(qc_data["cluster_"]).mean()
mean_rt = qc_data["rt"].groupby(qc_data["cluster_"]).mean()
qc_data["mean_mz"] = \
    (qc_data["mz"].groupby(qc_data["cluster_"])
     .apply(lambda x: x - mean_mz[x.name]))

qc_data["mean_rt"] = \
    (qc_data["rt"].groupby(qc_data["cluster_"])
     .apply(lambda x: x - mean_rt[x.name]))
# sss_data = sss_data.rename(columns={"area": "Area / 10e5 [au]"})

xvars = ["mean_mz", "mean_rt", "area"]
g = sns.PairGrid(data=qc_data,
                 y_vars=["Compound"],
                 x_vars=xvars,
                 hue="class_",
                 height=4)
g.map(sns.stripplot)
g.axes[0, 0].set_xlim(-0.01, 0.01)
g.axes[0, 1].set_xlim(-1, 1)
g.axes[0, 2].set_xticks(np.linspace(0, 3e5, 6))

t = g.axes[0, 2].get_xticks()
t = [str(x / 100000) for x in t ]
g.axes[0, 2].set_xticklabels(t);g.axes[0, 0].set_xlabel("Mean centered m/z")
g.axes[0, 1].set_xlabel("Mean centered Rt [s]")
g.axes[0, 2].set_xlabel("Area / $10^{5}$ [au]")
# g.savefig("rt_mz_area_qc_lpc.png", dpi=300)